In [1]:
import utils
import dao
import pandas as pd
import os
import datetime
import config
import json
import util_designite

In [2]:
CREATE_DATA_BASE = False
SHOW_LIST_OF_ANALYSED_COMMIT = False

In [3]:
# Conexao ao Banco de Dados de Commits do Cassandra

t1 = datetime.datetime.now()
os.system('clear')

lista_commits_entre_tags = []
print(f'Analise do Repositório de Código do Cassandra: intervalo de {config.tag1} até {config.tag2} - Data: {t1}\n')

if SHOW_LIST_OF_ANALYSED_COMMIT:
    lista_commits_entre_tags = utils.list_commits_between_tags(from_tag=config.tag1, to_tag=config.tag2, my_repository=config.repositorio_git_cassandra) 
    print(f'Para as tags: {config.tag1} e {config.tag2} é feita uma análise dos {len(lista_commits_entre_tags)} commits \
e para cada commit é guardada uma lista de arquivos modificados nesse commit.')
file_name = config.path_arquivos_modificados + '/' + f'lista_commits_entre_tags_{config.tag1}_{config.tag2}.txt'
utils.create_file_by_list_of_commits(file_name, lista_commits_entre_tags)

print(f'Configura e carrega o banco {dao.data_base}...')
if CREATE_DATA_BASE:
    # Recria o banco e insere todos os commits analisados
    try:
        dao.drop_tables()
        dao.create_tables()
        db_session = dao.create_session()
        commitsCompleteCollection = dao.CommitsCompleteCollection(session=db_session)
        filesCompleteCollection = dao.FilesCompleteCollection(session = db_session)
        print('Aguarde...')
        utils.save_complete_commits_and_modifield_files_in_db(lista_commits_entre_tags, commitsCompleteCollection, filesCompleteCollection)
        utils.display('save_complete_commits_and_modifield_files_in_db(')
        print('Commits e arquivos salvos com sucesso no banco!')
    except Exception as e:
        print(f'Erro: {e}')
else: 
    # Carrega o banco já criado
    db_session = dao.create_session()
    commitsCompleteCollection = dao.CommitsCompleteCollection(session=db_session)
    filesCompleteCollection = dao.FilesCompleteCollection(session = db_session)
    print(f'Banco {dao.data_base} carregado com sucesso! \n')

print('Aguarde...')
t2 = datetime.datetime.now()

delta = t2 - t1
print(f'Tempo de carregamento: {delta}')

Analise do Repositório de Código do Cassandra: intervalo de cassandra-3.0.0 até cassandra-3.11.11 - Data: 2021-12-06 09:23:44.203592

File: /Users/armandosoaressousa/git/msr/pilot-test/analises/modificados/v-3-11-11/lista_commits_entre_tags_cassandra-3.0.0_cassandra-3.11.11.txt created successfully
Configura e carrega o banco testecassandra.db...
Banco testecassandra.db carregado com sucesso! 

Aguarde...
Tempo de carregamento: 0:00:00.033271


In [4]:
# Carrega todos os commits e todos os arquivos em listas em memoria

all_commits = commitsCompleteCollection.query_all_commits()
all_files = filesCompleteCollection.query_all_files()

# Analise do esforço de manutencao dos Arquivos

In [5]:
# Gera um dataframe contendo a frequencia de linhas alteradas por arquivo

lista_linhas_por_file = []
for each in all_files:
    elemento = each.added_lines + each.deleted_lines
    lista_linhas_por_file.append(elemento)

lista_linhas_por_file.sort()

df_linhas_por_file = pd.Series(lista_linhas_por_file).value_counts().reset_index().sort_values('index').reset_index(drop=True)
df_linhas_por_file.columns = ['Lines', 'Frequency']
df_linhas_por_file.head(10)


,Lines,Frequency
0,1,291
1,2,1063
2,3,520
3,4,617
4,5,420
5,6,407
6,7,217
7,8,283
8,9,212
9,10,219


In [6]:
# Exporta o dataframe para o arquivo linhasporfile.csv

# Calcula os Quartis de linhas alteradas por arquivo

df_linhas_por_file.to_csv('linhasporfile.csv', index=False)
df_linhas_por_file.Lines.quantile([0.25,0.5,0.75, 1])

0.25      98.5
0.50     201.0
0.75     364.5
1.00    1852.0
Name: Lines, dtype: float64

In [7]:
# Ler o arquivo filesmodifiedlines.cvs originado de uma consulta do banco que mostra o arquivo e qtd de linhas alteradas

df_arquivos_linhas_modificadas = pd.read_csv('filemodifiedlines.csv')
lista_modifiedlines = df_arquivos_linhas_modificadas.modifiedlines.unique()

# Análise do esforço de manutencao dos Commits

In [8]:
lista_linhas_por_commit = []
for each in all_commits:
    lista_linhas_por_commit.append(each.lines)

lista_linhas_por_commit.sort()

df_linhas_por_commit = pd.Series(lista_linhas_por_commit).value_counts().reset_index().sort_values('index').reset_index(drop=True)
df_linhas_por_commit.columns = ['Commits', 'Frequency']
df_linhas_por_commit.head(10)

,Commits,Frequency
0,0,441
1,1,95
2,2,353
3,3,251
4,4,191
5,5,171
6,6,165
7,7,105
8,8,130
9,9,84


In [9]:
# Calcula os Quartis

df_linhas_por_commit.Commits.quantile([0.25,0.5,0.75, 1])

0.25      161.75
0.50      361.50
0.75      873.25
1.00    40365.00
Name: Commits, dtype: float64

In [10]:
# Exporta o arquivo linhasporcommit.csv contendo a quantidade de commits e frequencia

df_linhas_por_commit.to_csv('linhasporcommit.csv', sep=',', index=False)

In [11]:
%env CASSANDRA_PILOT_PATH=analises/designite/v-3-11-11/src/java/org/apache/cassandra
%env PILOT_PATH=analises/designite/v-3-11-11/src/java

!echo $CASSANDRA_PILOT_PATH
!echo $PILOT_PATH

env: CASSANDRA_PILOT_PATH=analises/designite/v-3-11-11/src/java/org/apache/cassandra
env: PILOT_PATH=analises/designite/v-3-11-11/src/java
analises/designite/v-3-11-11/src/java/org/apache/cassandra
analises/designite/v-3-11-11/src/java


In [12]:
!find $CASSANDRA_PILOT_PATH -name "*.java" > arquivosjava.txt

In [13]:
!find $CASSANDRA_PILOT_PATH -name *.java | xargs wc -l > locarquivosjava.txt
!tail -10 locarquivosjava.txt

     151 analises/designite/v-3-11-11/src/java/org/apache/cassandra/hints/Hint.java
     220 analises/designite/v-3-11-11/src/java/org/apache/cassandra/hints/HintsStore.java
      75 analises/designite/v-3-11-11/src/java/org/apache/cassandra/hints/CompressedHintsWriter.java
      85 analises/designite/v-3-11-11/src/java/org/apache/cassandra/hints/HintsDispatchTrigger.java
     260 analises/designite/v-3-11-11/src/java/org/apache/cassandra/hints/HintsBuffer.java
     132 analises/designite/v-3-11-11/src/java/org/apache/cassandra/hints/HintMessage.java
      28 analises/designite/v-3-11-11/src/java/org/apache/cassandra/hints/ChecksumMismatchException.java
      43 analises/designite/v-3-11-11/src/java/org/apache/cassandra/hints/package-info.java
     430 analises/designite/v-3-11-11/src/java/org/apache/cassandra/hints/HintsDescriptor.java
  314893 total


In [14]:
import os

CASSANDRA_PILOT_PATH = os.environ['CASSANDRA_PILOT_PATH']
PILOT_PATH = os.environ['PILOT_PATH']

print(CASSANDRA_PILOT_PATH)
print(PILOT_PATH)

analises/designite/v-3-11-11/src/java/org/apache/cassandra
analises/designite/v-3-11-11/src/java


In [15]:
# Quantidade de Arquivos .java encontrados no diretório src/java
!wc -l < arquivosjava.txt

    1520


In [16]:
# Mostra os 10 primeiros arquivos .java 
!head -10 arquivosjava.txt

analises/designite/v-3-11-11/src/java/org/apache/cassandra/metrics/LatencyMetrics.java
analises/designite/v-3-11-11/src/java/org/apache/cassandra/metrics/BufferPoolMetrics.java
analises/designite/v-3-11-11/src/java/org/apache/cassandra/metrics/MetricNameFactory.java
analises/designite/v-3-11-11/src/java/org/apache/cassandra/metrics/ThreadPoolMetrics.java
analises/designite/v-3-11-11/src/java/org/apache/cassandra/metrics/SEPMetrics.java
analises/designite/v-3-11-11/src/java/org/apache/cassandra/metrics/HintsServiceMetrics.java
analises/designite/v-3-11-11/src/java/org/apache/cassandra/metrics/DroppedMessageMetrics.java
analises/designite/v-3-11-11/src/java/org/apache/cassandra/metrics/CASClientRequestMetrics.java
analises/designite/v-3-11-11/src/java/org/apache/cassandra/metrics/ClientRequestMetrics.java
analises/designite/v-3-11-11/src/java/org/apache/cassandra/metrics/CompactionMetrics.java


In [17]:
# Cria uma lista com todos os arquivos .java do projeto cassandra

lista_arquivos_projeto = []
with open('arquivosjava.txt', mode='r+', encoding='utf-8') as file:
  for line in file:  
    line = line.rstrip()
    line = line.replace(PILOT_PATH, '')
    line = line.replace('/', '.')
    lista_arquivos_projeto.append(line)

In [18]:
lista_arquivos_projeto = []

with open('arquivosjava.txt', mode='r+', encoding='utf-8') as file:
  for line in file:  
    line = line.rstrip()
    line = line.replace(PILOT_PATH, '')
    line = line.replace('/', '.')
    lista_arquivos_projeto.append(line)

In [19]:
def extrai_apenas_nome_arquivo(my_file):
  string_temp = my_file #'org.apache.cassandra.repair.Validator.java'
  split1 = string_temp.split('.')
  nome = split1[(len(split1)-2)] + '.' + split1[(len(split1)-1)]
  return nome


In [20]:
filename_commits = 'dicionario_commits_com_arquivos_java_modificados.json'
jsonfile_commits = None
with open(filename_commits) as f:
  jsonfile_commits = json.load(f)
dicionario_commits_com_arquivos_java_modificados_from_v_3_0_0_to_v_3_11_11 = jsonfile_commits

filename_files = 'dicionario_files_com_commits.json'
jsonfile_files = None
with open(filename_files) as f:
  jsonfile_files = json.load(f)
dicionario_files_com_commits_from_v_3_0_0_to_v_3_11_11 = jsonfile_files

print(f'Total de commits analisados da versão {config.tag1} a {config.tag2}: { len(dicionario_commits_com_arquivos_java_modificados_from_v_3_0_0_to_v_3_11_11) }')
print('')
print(f'Total de arquivos analisados da versão {config.tag1} a {config.tag2}: { len(dicionario_files_com_commits_from_v_3_0_0_to_v_3_11_11) }')

Total de commits analisados da versão cassandra-3.0.0 a cassandra-3.11.11: 5560

Total de arquivos analisados da versão cassandra-3.0.0 a cassandra-3.11.11: 1926


In [21]:
dicionario_arquivo_frequencia_commits = {}

for arquivo in lista_arquivos_projeto:
  nome_arquivo = extrai_apenas_nome_arquivo(arquivo)
  if nome_arquivo in dicionario_files_com_commits_from_v_3_0_0_to_v_3_11_11:
    quantidade_commits = len(dicionario_files_com_commits_from_v_3_0_0_to_v_3_11_11[nome_arquivo])
    dicionario_arquivo_frequencia_commits[nome_arquivo] = quantidade_commits

print(len(dicionario_arquivo_frequencia_commits) )

1148


In [22]:
# Mostra o dicionário arquivos do projeto com a quantidade de commits ordenado descendente

import operator

dicionario_arquivo_frequencia_commits_ordenado = dict( sorted(dicionario_arquivo_frequencia_commits.items(), key=operator.itemgetter(1),reverse=True)) 
print(f'Total de dicionario_arquivo_frequencia_commits_ordenado: {len(dicionario_arquivo_frequencia_commits_ordenado)}')

Total de dicionario_arquivo_frequencia_commits_ordenado: 1148


In [23]:
# Dataframe contendo detalhes dos 100 arquivos mais modificados

df_arquivos_modificados_v_3_11_11 = util_designite.init_df_100_arquivos_mais_modificados(versao='3.11.11', path='analises/modificados/v-3-11-11')
df_arquivos_modificados_v_3_11_11.sort_values(['name', 'date', 'hash']).head(3)

,Unnamed: 0,index,name,hash,date,added_lines,deleted_lines,modifications,old_path,new_path,change_type,nloc,complexity
1337,1337,1337,AbstractCluster.java,c100f1ac40137a0d6d0793930cdfce3845880a12,2018-12-18 14:20:52,406,0,406,NaN,test/distributed/org/apache/cassandra/distribu...,ADD,287.0,55.0
1338,1338,1338,AbstractCluster.java,a7d8ba7b10a441f9710724e65a939a46add0ae78,2019-02-11 15:37:07,20,10,30,test/distributed/org/apache/cassandra/distribu...,test/distributed/org/apache/cassandra/distribu...,MODIFY,297.0,55.0
1339,1339,1339,AbstractCluster.java,7d2c3c215f65ee41f86886304257647fc24b1f70,2019-04-04 14:39:57,17,1,18,test/distributed/org/apache/cassandra/distribu...,test/distributed/org/apache/cassandra/distribu...,MODIFY,311.0,59.0


In [24]:
# DataFrame contendo os 100 arquivos com maior frequencia de commits

df_100_arquivos_maior_frequencia_commits = df_arquivos_modificados_v_3_11_11.copy()

df_100_arquivos_maior_frequencia_commits.query("name=='DatabaseDescriptor.java'").sort_values(by=['date']).head(3)

,Unnamed: 0,index,name,hash,date,added_lines,deleted_lines,modifications,old_path,new_path,change_type,nloc,complexity
242,242,242,DatabaseDescriptor.java,b11fba750c610de5e97acba070cc571cf0a96416,2015-03-20 08:02:39,6,0,6,src/java/org/apache/cassandra/config/DatabaseD...,src/java/org/apache/cassandra/config/DatabaseD...,MODIFY,1597.0,410.0
233,233,233,DatabaseDescriptor.java,47341eb6aaca318d0ffc0e9f906b98db50b9e9ff,2015-07-30 14:03:17,17,1,18,src/java/org/apache/cassandra/config/DatabaseD...,src/java/org/apache/cassandra/config/DatabaseD...,MODIFY,1445.0,374.0
238,238,238,DatabaseDescriptor.java,e2c6341898fa43b0e262ef031f267587050b8d0f,2015-08-03 10:12:47,0,3,3,src/java/org/apache/cassandra/config/DatabaseD...,src/java/org/apache/cassandra/config/DatabaseD...,MODIFY,1584.0,407.0


In [25]:
df_arquivo_frequencia_commits_ordenado = pd.DataFrame.from_dict(dicionario_arquivo_frequencia_commits_ordenado, orient='index', columns=['Total_de_Commits'])
df_arquivo_frequencia_commits_ordenado

,Total_de_Commits
StorageService.java,144
ColumnFamilyStore.java,89
DatabaseDescriptor.java,87
CompactionManager.java,74
SSTableReader.java,53
StorageProxy.java,53
Config.java,52
CassandraDaemon.java,51
SelectStatement.java,44
LegacyLayout.java,42


In [26]:
lista_commits_df_arquivo_frequencia_commits_ordenado = df_arquivo_frequencia_commits_ordenado['Total_de_Commits'].to_list()
list_of_content = lista_commits_df_arquivo_frequencia_commits_ordenado

df_frequencia_commits = pd.Series(list_of_content).value_counts().reset_index().sort_values('index').reset_index(drop=True)
df_frequencia_commits.columns = ['Commits', 'Frequency']
df_frequencia_commits

,Commits,Frequency
0,1,359
1,2,222
2,3,119
3,4,99
4,5,74
5,6,51
6,7,38
7,8,28
8,9,27
9,10,15


In [27]:
df_frequencia_commits.Commits.quantile([0.25,0.5,0.75, 1])

0.25     11.25
0.50     21.50
0.75     36.75
1.00    144.00
Name: Commits, dtype: float64